In [1]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D 
from keras import optimizers
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
K.set_image_dim_ordering('th')
import matplotlib.pyplot as plt

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def cnn_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(3, 150, 150)))
    model.add(Activation('relu'))

    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))

    model.add(Conv2D(64,(3, 3)))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(12))
    model.add(Activation('softmax'))

    # Compile model
    rmsprop = optimizers.RMSprop(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 
    return model

In [3]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rotation_range=180, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   vertical_flip=True,
                                   fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'CAX_Superhero_Train_Original/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'CAX_Superhero_Train_Original/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 3800 images belonging to 12 classes.
Found 1633 images belonging to 12 classes.


In [4]:
early_stopping_monitor = EarlyStopping(monitor='val_acc', patience=10, verbose=1)
# build the model
model = cnn_model()

model.fit_generator(train_generator, 
                    steps_per_epoch=3800//batch_size, 
                    epochs=50, 
                    validation_data=validation_generator, 
                    validation_steps=1633//batch_size, 
                    callbacks=[early_stopping_monitor], 
                    verbose=1)

Epoch 1/50
118/118 [==============================] - 27s 228ms/step - loss: 2.3577 - acc: 0.2458 - val_loss: 2.1465 - val_acc: 0.2604
Epoch 2/50
118/118 [==============================] - 27s 226ms/step - loss: 1.9912 - acc: 0.3542 - val_loss: 1.9655 - val_acc: 0.3685
Epoch 3/50
118/118 [==============================] - 26s 224ms/step - loss: 1.8813 - acc: 0.4053 - val_loss: 1.9646 - val_acc: 0.3941
Epoch 4/50
118/118 [==============================] - 26s 224ms/step - loss: 1.7619 - acc: 0.4405 - val_loss: 1.7698 - val_acc: 0.4235
Epoch 5/50
118/118 [==============================] - 26s 218ms/step - loss: 1.7254 - acc: 0.4576 - val_loss: 1.7251 - val_acc: 0.4360
Epoch 6/50
118/118 [==============================] - 26s 219ms/step - loss: 1.6642 - acc: 0.4788 - val_loss: 1.7579 - val_acc: 0.4428
Epoch 7/50
118/118 [==============================] - 26s 222ms/step - loss: 1.6570 - acc: 0.4830 - val_loss: 1.7251 - val_acc: 0.4379
Epoch 8/50
118/118 [==============================] - 2

In [6]:
datagen = ImageDataGenerator(rescale=1./255)

generator = datagen.flow_from_directory(
        'CAX_Superhero_Test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)  # keep data in same order as labels

Found 3375 images belonging to 1 classes.


In [7]:
probabilities = model.predict_generator(generator, verbose=1)

106/106 [==============================] - 6s 56ms/step


In [8]:
y_classes = probabilities.argmax(axis=-1)

In [9]:
predictions = []
for i in range(0, len(y_classes)):    
    for hero, cls in validation_generator.class_indices.iteritems():
        if cls == y_classes[i]:
            predictions.append([generator.filenames[i], hero])

In [10]:
temp_submission = pd.DataFrame(data=predictions, 
                          columns=['filename', 'Superhero'])

In [11]:
temp_submission['filename'] = temp_submission['filename'].str.slice(5)
temp_submission['filename'] = temp_submission['filename'].str.slice(0,-4)

In [12]:
submission = pd.read_csv('Superhero_Submission_Format.csv')

In [13]:
submission = submission.drop(['Superhero'], axis=1)

In [14]:
submission = pd.merge(submission, temp_submission,
              on='filename') 

In [15]:
submission.Superhero.unique()

array(['spider_man', 'super_man', 'iron_man', 'black_panther', 'aqua_man',
       'bat_man', 'captain_america', 'cat_woman', 'hulk', 'avengers',
       'ant_man', 'ghostrider'], dtype=object)

In [16]:
submission.to_csv('submission.csv', index=False)